In [ ]:
from repo_checker import *
from issue_formater import *
from script_review import *

In [ ]:
import pandas as pd

In [ ]:
challenges_data = load_challenges("retos_data.json")

In [ ]:
df_repos = pd.read_csv("data/students.csv")

In [ ]:
test_file_path = "test_data.json"
test_file = load_tests(test_file_path) 

In [ ]:
template_path = "templates/oop_retos.md"

In [ ]:
parent_repo = "python-la-paz/psg-retos"

In [ ]:
for index, row in df_repos.iterrows():
    owner, repo_name, url = format_url(row["url"])
    print(f"Reviewing: {owner}/{repo_name}")
    repo = get_repo(owner, repo_name)
    fork_status = check_fork(repo, parent_repo)
    branch_status = check_branch_exists(repo, owner)
    activity_status = check_activity(repo, owner)
    no_code_status = [fork_status, branch_status, activity_status]
    for challenge in challenges_data:
        issue_title = challenge["title"]
        file_paths = challenge["paths"]
        max_date = challenge["max_date"]
        
        files_status = []
        code_review = []
        dates_review = []
        try:
            if not check_issue_exists(repo, issue_title):
                for path in file_paths:
                    status = check_file_exists(repo, path, branch=owner)
                    code = get_code_from_file(repo, path, branch=owner) if status else ""
                    files_status.append(status)
                    test = [t for t in test_file if t['file'] == path][0]
                    if status:
                        dates_review.append(check_commit_dates(repo, path, max_date, branch=owner))
                        
                    else:
                        dates_review.append(False)
                    code_review.append(run_tests(code, test))
                
        except Exception as e:
            print(f"Error processing {issue_title}: {e}")
        issue_body = summary_code_review(owner,files_status,dates_review,no_code_status,code_review,challenge,template_path)
        with open(f"reviews/{owner}_review.md", "w", encoding="utf-8") as f:
            f.write(issue_body)